### imports

In [ ]:
! pip install Unidecode
#!python -m spacy download pt
! pip install python-telegram-bot --upgrade
! pip install tflearn
! pip install pypsexec
import requests
from unidecode import unidecode
import datetime
import pickle
#import spacy
from telegram import KeyboardButton
from telegram import Bot
from telegram import ReplyKeyboardMarkup
import re
import os
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tensorflow as tf
import tflearn
import random
import json
from tensorflow.python.framework import ops

import sys
import string
from email.mime.text import MIMEText
from subprocess import Popen, PIPE
from pypsexec.client import Client

nltk.download('punkt')
stemmer = LancasterStemmer()
#nlp = spacy.load('pt')

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (13.1)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path_dev = '/content/drive/My Drive/bot/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###json

In [ ]:
dados={
    "intents": [
        { 
            "tag": "saudacao",
            "patterns": ["Olá", "olaaa", "Oi", "Bom dia", "Boa tarde", "Boa noite", "Opa", "E aí", "Oi bot", "Olá bot", "alguém aí?"],
            "respostas_dia": ["Bom dia nome, alguma requisição ou incidente?", "Olá nome, como te ajudo?", "Bom dia nome, pois não?"],
            "respostas_tarde": ["Boa tarde nome, alguma requisição ou incidente?", "Olá nome, como te ajudo?", "Boa tarde nome, pois não?"],
            "respostas_noite": ["Boa noite nome, alguma requisição ou incidente?", "Olá nome, como te ajudo?", "Boa noite nome, pois não?"],
            "contexto": ["Hora", "Nome da Pessoa"]
        },
        {
            "tag": "incidente",
            "patterns": ["Tenho um incidente", "incidente", "Estou com um incidente", "Problema de incidente", "Estou com um problema", "sim, incidente"],
            "responses": ["Entendi, me informe seu incidente que tentarei te ajudar"]
        },
        {
            "tag": "requisicao",
            "patterns": ["Tenho uma requisicao", "Quero pedir algo", "Requisicao", "sim, uma requisição", "Quero solicitar algo", "Quero pedir algo",\
                         "Tenho uma requisicao para fazer", "Tenho um pedido", "Quero fazer uma requisição"],
            "responses": ["Entendi, faça sua requisição que tentarei te ajudar"]
        },
        {
            "tag": "encerramento",
            "patterns": ["tchau", "valeu ai", "deu certo, valeu", "deu certo, tchau", "até a próxima", "vou sair", "to indo nessa", "saindo aqui",
                         "tchauzinho", "desta", "desisti, quero cancelar", "cancelar operacao", "obrigado", "obg"],
            "respostas_dia": ["Então até a próxima nome!", "Tchauzinho nome e bom dia!", "Até a próxima nome, tenha um bom dia!"],
            "respostas_tarde": ["Então até a próxima nome!", "Tchauzinho nome e boa tarde!", "Até a próxima nome, tenha uma boa tarde!"],
            "respostas_noite": ["Então até a próxima nome!", "Tchauzinho nome e boa noite!", "Até a próxima nome, tenha uma boa noite!"],
            "contexto": ["Hora", "Nome da Pessoa"]
        },
        {
            "tag": "afirmacao",
            "patterns": ["Sim", "ok", "sim, por favor", "sim, obrigado", "Quero sim", "yes"],
            "responses": ["Ficamos por aqui?"]
        },
        {
            "tag": "negacao",
            "patterns": ["não resolveu", "não", "infelizmente não"],
            "responses": ["Neste caso não tenho nenhuma ação automática. Vou te passar para um técnico."]
        }
   ]
}

In [ ]:
dados_inc={
    "intents": [
          {
              "Tag": "01_Impressora",
              "Pergunta": ["Não estou conseguindo imprimir", "Problemas com impressora", "Impressora não imprime", "Impressora não funciona"],
              "RespostaSelfService": ["Verifique se o nobreak/estabilizador do equipamento está ligado e se a impressora está ligada! Verifique também  se a impressora selecionada para impressão foi a correta e se persistir, desligue e religue a impressora e tente novamente. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "02_Internet",
              "Pergunta": ["Não consigo acessar a internet", "Problema com wifi", "Problema com a internet", "Sem internet", "Internet não funciona"],
              "RespostaSelfService": ["Verifique se o cabo de rede está devidamente plugado no equipamento, feche todos os navegadores, abre novamente um deles e tente acessar um site diferente e se o problema persistir, reinicie seu computador e tente navegar novamente. Com as orientações repassadas, o seu problema foi resolvido? "],
              "TextoAutomacao": ""
          },
          {
              "Tag": "03_Notebook",
              "Pergunta": ["Não consigo ligar o notebook", "Problema com notebook", "Notebook não funciona"],
              "RespostaSelfService": ["Verifique se o cabo de força está ligado ao notebook e o Led de funcionamento/carregamento está acesso, se o Led de funcionamento/carregamento não acender, plugue o carregador em outra tomada e segure o botão liga/desliga do notebook por 3 segundos, para ligar/desligar novamente o equipamento. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "04_Computador",
              "Pergunta": ["Não consegui ligar o computador", "Problema com computador", "tela azul", "windows não inicia" ,"computador não liga"],
              "RespostaSelfService": ["Verifique se o nobreak/estabilizador do equipamento está ligado, verifique também se o cabo de força do equipamento está conectado ao nobreak/estabilizador e se o monitor do computador está ligado. Se não der certo, segure o botão liga/desliga do computador por 3 segundos, para ligar/desligar novamente o equipamento. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "05_Ponto",
              "Pergunta": ["Não consegui registrar o ponto", "Problema no relógio de ponto", "Ponto registrado errado", "Ponto errado"],
              "RespostaSelfService": ["Após a batida, o ponto leva cerca de 15 minutos para atualizar na Área Restrita, por favor aguarde! Após esse período acesse a Área Restrita, com seu CPF e senha, clique em Meu Ponto -> Imprimir Cartão de Ponto, caso a batida não seja exibida na data especificada talvez seja necessário realizar uma justificativa de ponto. Para isso, acesse Meu Ponto -> Justificativa de Ponto, selecione um Motivo, acessando a Ajuda ao lado caso necessário, preencha as observações pertinentes, e clique em Incluir. Uma vez que o avaliador do ponto aprovar sua justificativa, seu ponto estará normalizado"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "06_Celular",
              "Pergunta": ["Celular não conecta no wifi", "Wifi no celular", "Configurar wifi no celular"],
              "RespostaSelfService": ["Verifique se Wifi do celular está ativado, acesse as configurações de Wifi do seu celular, procure pela rede TCE_Corporativo, e clique nesssa rede. No campo Usuário, insira seu CPF, no campo Senha, insira sua senha da Área Restrita, no campo Certificado, selecione Não Validar, em seguida clique no botão Conectar para acessar o Wifi Corporativo no seu smartphone. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "07_Nobreak",
              "Pergunta": ["Nobreak não liga", "Nobreak não funciona", "Nobreak bipando"],
              "RespostaSelfService": ["Verifique se o cabo de força do equipamento está conectado a rede elétrica, conecte o cabo a rede elétrica, acione o botão de ligar novamente, alguns modelos necessitam que o botão seja pressionado/segurado por 3 segundos até que se escute um bip de acionamento. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "08_Mouse",
              "Pergunta": ["Mouse não funciona", "Mouse não movimente", "Mouse não responde"],
              "RespostaSelfService": ["Verifique se o cabo do mouse está conectado à porta USB do seu computador/notebook, conecte o cabo ao computador, se já estiver conectado, remova o cabo e o reconecte a uma porta USB diferente. Se o problema persistir, reinicie seu computador/notebook. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "09_Teclado",
              "Pergunta": ["Teclado não funciona", "Teclado não digitam", "Teclado não responde"],
              "RespostaSelfService": ["Verifique se o cabo do Teclado está conectado à porta USB do seu computador/notebook, conecte o cabo ao computador, se já conectado, remova o cabo e o reconecte a uma porta USB diferente. Se o problema persistir reinicie seu computador/notebook. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          },
          {
              "Tag": "10_Logar",
              "Pergunta": ["Não estou conseguindo logar", "Problema ao logar", "Problema no login"],
              "RespostaSelfService": ["Digite o CPF de seu usuário e o mesmo será debloqueado em instantes."],
              "TextoAutomacao": ["O logon será debloqueado em instantes, tente acessar em alguns minutos ok?"]
          },
          {
              "Tag": "11_Papel impressora",
              "Pergunta": ["O papel atolou", "O papel ficou preso", "Papel enganxou"],
              "RespostaSelfService": ["Remova o papel, desligue e ligue novamente a impressora. Com as orientações repassadas, o seu problema foi resolvido?"],
              "TextoAutomacao": ""
          }
   ]
}


In [ ]:
dados_req={
    "intents": [
        {
            "Tag": "R01_Trocar_senha",
            "Pergunta": ["Gostaria de trocar minha senha", "Pode trocar minha senha", "Como eu troco minha senha"],
            "RespostaSelfService": ["Acesse o endereço Https://arearestrita.tce.rn.gov.br, informe seu CPF e senha, use o menu Ferramentas / Segurança / Alterar Senha, digite seu senha atual, digite sua  nova senha, confirme sua nova senha e clique no botão Alterar no canto inferior direto da tela. Com as orientações repassadas, o seu problema foi resolvido?"],
            "TextoAutomacao": "Posso ressetar sua senha e gerar uma senha automática e encaminhar para seu e-mail?"
        },
        {
            "Tag": "R02_Desbloquear_usuario",
            "Pergunta": ["Usuário bloqueado", "Problema ao logar com meu usuário", "Meu usuário não consegue logar"],
            "RespostaSelfService": ["Digite o CPF de seu usuário e o mesmo será debloqueado em instantes."],
            "TextoAutomacao": "O logon do CPF será debloqueado em instantes, tente acessar em alguns minutos. "
        },
		    {
            "Tag": "R03_Acessar área restrita",
            "Pergunta": ["Não consigo acessar a Área Restrita", "não acessso área restrita", "Quero acessar a Área restrita"],
            "RespostaSelfService": ["Feche todos os navegadores abertos, em seguida abra novamente um navegador e acesse o endereço Https://arearestrita.tce.rn.gov.br, informe o CPF do usuário e a sua senha. Caso o problema persista, seu usuário e/ou senha podem estar bloqueados, solicite então o desbloqueio do mesmo. Com as orientações repassadas, o seu problema foi resolvido?"],
            "TextoAutomacao": ""
        },
		    {
            "Tag": "R04_Uso de certificado digital",
            "Pergunta": ["Não consigo usar o certificado digital", "Não consigo assinar um documento", "desejo assinar com certificado digital", "assinar eletrônicamente", "certificado digital"],
            "RespostaSelfService": ["Insira seu Token na porta USB do seu computador, em seguida verifique se o ícone do ICPBravoAcess encontra-se visível no canto superior direito do seu navegador. Clique com o botão esquerdo do mouse sobre esse ícone e verifique se seu Certificado foi exibido na lista, uma vez exibido, tente realizar novamente utilizar o certificado. Com as orientações repassadas, o seu problema foi resolvido?"],
            "TextoAutomacao": ""
        },
		    {
            "Tag": "R05_Consultar Ponto",
            "Pergunta": ["Não consigo consultar meu ponto", "Não vejo meu banco de horas", "onde acesso meu banco de horas"],
            "RespostaSelfService": ["Para verificar o banco de horas, acesse o endereço Https://arearestrita.tce.rn.gov.br, informe o CPF do usuário e a sua senha, e em seguida acesse Administrativo / Ponto - Controle de Ponto / Meu Banco de Horas. Nessa tela serão exibidos o banco de horas do usuário, o saldo do mês, e do trimestre. Ainda nessa tela é possível selecionar o trimestre que se deseja consultar. Também é possível verificar o ponto diretamente na Área Restrira através da Mesa Eletrônica e clicando-se no ícone Meu Ponto, onde serão exibidos, além do banco de horas, as batidas diárias do ponto do mês atual. Com as orientações repassadas, o seu problema foi resolvido?"],
            "TextoAutomacao": ""
        },
		    {
            "Tag": "R06_Consultar Contra-Cheque",
            "Pergunta": ["Não consigo consultar meu Contra-Cheque", "Onde acesso meu Contra-Cheque"],
            "RespostaSelfService": ["Para verificar o Contra-Cheque, acesse o endereço Https://arearestrita.tce.rn.gov.br, informe o CPF do usuário e a sua senha, e em seguida acesse Administrativo / Recursos Humanos / Contracheque. Nessa tela será necessário informar a competência desejada. Uma vez informados o mês e o ano, deve clicar no botão Gerar Contracheque, onde um arquivo pdf com o contracheque da competência informada será disponbilizado. Com as orientações repassadas, o seu problema foi resolvido?"],
            "TextoAutomacao": ""
        },
		    {
            "Tag": "R07_Acesso a funcionalidade",
            "Pergunta": ["Não possuo a funcionalidade", "Como solicito acesso a uma funcionalidade", "funcionalidade não disponível", "Você não tem permissão"],
            "RespostaSelfService": ["Caso uma funcionalidade esteja indisponível para seu perfil ou a mensagem, Você não tem permissão, seja exibida, se faz necessário que seu superior imediato, ou Diretor, faça a solicitação da mesma para você. Uma vez logado na Área Restrita, ele deve acessar Ferramentas / Segurança / Solicitação de acesso às Funcionalidades. Nessa tela deve-se informar o CPF do usuário ao qual se deseja conceder os acessos, e em seguida clicar no botão Consultar, no canto inferior direito. Serão então exibidos as informações do funcionário bem como uma lista com todas as funcionalidade da Área Restrita, onde se deve elencar as funcionaliade desejadas, selecionando-as. Ao final das seleções, clicar no botão Incluir, no canto inferior direito. Em poucos minutos as funcionalidades estarão disponível para o usuário."],
            "TextoAutomacao": ""
        }		
   ]
}

### Ações

#### change pass

In [ ]:
def change_password(celular):
  erro = 0
  mensagemErro = ""

  if not celular:
      erro = 1
      mensagemErro = "Esse script deve ser executado passando um celular de parâmetro."

  if erro == 0:
      try:
          # cria uma conexão criptografada com o AD
          c = Client("ad", username="chatbot", password="P@ssw0rd")

          # conecta
          c.connect()
      except:
          erro = 2
          mensagemErro = "Não foi possível se conectar ao servidor."

  if erro == 0:
      try:
          c.create_service()
      except:
          erro = 3
          mensagemErro = "Não foi possível criar o serviço do servidor."

  if erro == 0:
      try:
          stdout, stderr, rc = c.run_executable("cmd.exe", arguments='/c powershell "Get-AdUser -Filter * -Properties OfficePhone,mail | Select-Object OfficePhone,SamAccountName,mail | Where-Object OfficePhone -like '+celular)
          if not stdout:
              erro = 4
              mensagemErro = "O solicitante não possui o número do celular cadastrado no servidor."
          else:
              listaRetorno = stdout.decode('utf-8').strip().split("\r\n")
              if len(listaRetorno) < 3:
                  erro = 5
                  mensagemErro = "Erro na tentativa de pegar o solicitante usando o celular cadastrado no servidor. listaRetorno menor que três."
              else:
                  usuario = listaRetorno[2]
                  while usuario.find("  ") != -1:
                      usuario = usuario.replace("  ", " ")
                  dadosUsuario = usuario.split(" ")
                  if len(dadosUsuario) < 3:
                      erro = 6
                      mensagemErro = "O solicitante não possui o e-mail cadastrado no servidor."
                  else:
                      celular = dadosUsuario[0]
                      contaAD = dadosUsuario[1]
                      email = dadosUsuario[2]
      except:
          erro = 7
          mensagemErro = "Não foi possível consultar o usuário no servidor."

  if erro == 0:
      try:
          novaSenha = random.choices(string.ascii_uppercase)+random.choices(string.ascii_uppercase)+random.choices(string.ascii_uppercase)+random.choices(string.ascii_uppercase)+random.choices(string.ascii_lowercase)+random.choices(string.ascii_lowercase)+random.choices(string.ascii_lowercase)+random.choices(string.ascii_lowercase)+random.choices(string.digits)+random.choices(string.digits)
          random.shuffle(novaSenha)
          novaSenha = ''.join([str(elem) for elem in novaSenha])

          stdout1, stderr1, rc1 = c.run_executable("cmd.exe", arguments="/c powershell Enable-ADAccount -Identity "+contaAD)
          stdout2, stderr2, rc2 = c.run_executable("cmd.exe", arguments="/c powershell Unlock-ADAccount -Identity "+contaAD)
          stdout3, stderr3, rc3 = c.run_executable("cmd.exe", arguments='/c powershell Set-ADAccountPassword -Identity '+contaAD+' -Reset -NewPassword (ConvertTo-SecureString -AsPlainText "'+novaSenha+'" -Force)')

          if stderr3:
              erro = 8
              mensagemErro = "Erro na execução da troca de senha."
          else:
              msg = MIMEText("Prezado usuário, foi enviado um email para "+email+". A sua senha foi modificada para "+novaSenha+" e sua conta já pode ser utilizada novamente. Equipe de suporte da diretoria de informática do TCE/RN.")
              msg["From"] = "chatbox@tcern.tc.br"
              msg["To"] = "root@chatbot.tcern.tc.br"
              msg["Subject"] = "Mudança de senha efetuada"
              p = Popen(["/usr/sbin/sendmail", "-t", "-oi"], stdin=PIPE)
              p.communicate(msg.as_bytes())
      except:
          erro = 9
          mensagemErro = "Não foi possível desbloquear o usuário."

  if erro > 3 or erro == 0:
      c.remove_service()
  if erro > 2 or erro == 0:
      c.disconnect()

  return mensagemErro


#### unlock

In [ ]:
def unlock_user(celular):
  erro = 0
  mensagemErro = ""

  if not celular:
    erro = 1
    mensagemErro = "Esse script deve ser executado passando um celular de parâmetro."

  if erro == 0:
      try:
          # cria uma conexão criptografada com o AD
          c = Client("ad", username="chatbot", password="P@ssw0rd")

          # conecta
          c.connect()
      except:
          erro = 2
          mensagemErro = "Não foi possível se conectar ao servidor."

  if erro == 0:
      try:
          c.create_service()
      except:
          erro = 3
          mensagemErro = "Não foi possível criar o serviço do servidor."

  if erro == 0:
      try:
          stdout, stderr, rc = c.run_executable("cmd.exe", arguments='/c powershell "Get-AdUser -Filter * -Properties OfficePhone,mail | Select-Object OfficePhone,SamAccountName,mail | Where-Object OfficePhone -like '+celular)
          if not stdout:
              erro = 4
              mensagemErro = "O solicitante não possui o número do celular cadastrado no servidor."
          else:
              listaRetorno = stdout.decode('utf-8').strip().split("\r\n")
              if len(listaRetorno) < 3:
                  erro = 5
                  mensagemErro = "Erro na tentativa de pegar o solicitante usando o celular cadastrado no servidor. listaRetorno menor que três."
              else:
                  usuario = listaRetorno[2]
                  while usuario.find("  ") != -1:
                      usuario = usuario.replace("  ", " ")
                  dadosUsuario = usuario.split(" ")
                  if len(dadosUsuario) < 2:
                      erro = 6
                      mensagemErro = "Erro na tentativa de pegar o solicitante usando o celular cadastrado no servidor. dadosUsuarios menor que três."
                  else:
                      celular = dadosUsuario[0]
                      contaAD = dadosUsuario[1]
                      email = ""
                      if (len(dadosUsuario) > 2):
                          email = dadosUsuario[2]
      except:
          erro = 7
          mensagemErro = "Não foi possível consultar o usuário no servidor."

  if erro == 0:
      try:
          stdout, stderr, rc = c.run_executable("cmd.exe", arguments="/c powershell Enable-ADAccount -Identity "+contaAD)
          stdout, stderr, rc = c.run_executable("cmd.exe", arguments="/c powershell Unlock-ADAccount -Identity "+contaAD)
      except:
          erro = 8
          mensagemErro = "Não foi possível desbloquear o usuário."

  if erro > 3 or erro == 0:
      c.remove_service()
  if erro > 2 or erro == 0:
      c.disconnect()

  return [mensagemErro]


### Telegram

In [ ]:
class TelegramBot():

    def __init__(self, dados, dados_req, dados_inc):
        token = ''
        self.url_base = f'https://api.telegram.org/bot{token}/'
        #self.bot = Bot(token=token, base_url=self.url_base)
        self.dados = dados
        self.dados_req = dados_req
        self.dados_inc = dados_inc
        self.contextos = {
                          "Nome": "",
                          "Tempo": ""
                        }
        self.estados = {
                          "saudacao": True,
                          "incidente": False,
                          "requisicao": False,
                          "encerramento": False
                        }
        self.model_req, self.words_req, self.labels_req = self.carregar_dados("Requisicao")
        self.model_inc, self.words_inc, self.labels_inc = self.carregar_dados("Incidentes")
        self.model, self.words, self.labels = self.carregar_dados("model")

    def Iniciar(self):
        '''
        '''
        update_id = None
        self.contextos['Tempo'] = self.obtem_tempo()
        while True:
            atualizacao = self.obter_novas_mensagens(update_id)
            dados = atualizacao["result"]
            if dados:
                for dado in dados:
                    update_id = dado['update_id']
                    mensagem = str(dado["message"]["text"])
                    self.contextos['Nome'] = str(dado["message"]["from"]["first_name"])
                    chat_id = dado["message"]["from"]["id"]
                    resposta = self.criar_resposta(mensagem)
                    self.responder(resposta, chat_id)

    def obter_novas_mensagens(self, update_id):
        '''
        '''
        link_requisicao = f'{self.url_base}getUpdates?timeout=100'
        if update_id:
            link_requisicao = f'{link_requisicao}&offset={update_id + 1}'
        resultado = requests.get(link_requisicao)
        return json.loads(resultado.content)
    
    def obtem_status(self, tag):
      '''
      '''
      if tag == 'requisicao':
        self.estados['requisicao'] = True
      elif tag == 'incidente':
        self.estados['incidente'] = True
      for key, value in self.estados.items():
        if value:
          return key

    def muda_status(self):
      for key, value in self.estados.items():
        if value:
          self.estados[key] = False
        self.estados['saudacao'] = True
        break

    def decide_respostas_tempo(self, tg):
      '''
      '''
      if self.contextos['Tempo'] == 'manha':
          responses = tg['respostas_dia']
      elif self.contextos['Tempo'] == 'tarde':
        responses = tg['respostas_tarde']
      else:
        responses = tg['respostas_noite']
      return responses


    def criar_resposta(self, mensagem):
          '''
          '''
          results = self.model.predict([self.bag_of_words(unidecode(mensagem), self.words)])
          idx = np.argmax(results)
          tag = self.labels[idx]
          status = self.obtem_status(tag) 
          print(status)
          
          #modelo principal
          if results[0][idx] > 0.7:
            if status == tag or tag=='afirmacao' or tag=='negacao':                
              if tag == 'saudacao':
                self.estados['saudacao'] = False
                intencao = [intent for intent in self.dados['intents'] if intent['tag']== 'saudacao']
                responses = self.decide_respostas_tempo(intencao[0])
                return random.choice(responses).replace('nome',self.contextos['Nome'])
              elif tag == 'incidente':
                responses = [intent['responses'] for intent in self.dados['intents'] if intent['tag']== 'incidente']                
                return random.choice(responses[0])
              elif tag == 'afirmacao':
                if self.estados['incidente']:
                  self.estados['incidente'] = False
                  self.estados['encerramento'] = True
                  responses = [intent['responses'] for intent in self.dados['intents'] if intent['tag'] == 'afirmacao']
                  return random.choice(responses[0])
                elif self.estados['encerramento']:
                  self.muda_status()
                  intencao = [intent for intent in self.dados['intents'] if intent['tag'] == 'encerramento']
                  responses = self.decide_respostas_tempo(intencao[0])
                  return random.choice(responses).replace('nome',self.contextos['Nome'])
              elif tag == 'negacao':
                if self.estados['incidente']:
                  self.estados['incidente'] = False
                  self.estados['encerramento'] = True
                  responses = [intent['responses'] for intent in self.dados['intents'] if intent['tag']== 'negacao']
                  return random.choice(responses[0])
                elif self.estados['encerramento']:
                  self.estados['encerramento'] = False
                  self.estados['incidente'] = False
                  self.estados['requisicao'] = False
                  responses = ['Mais um incidente ou uma requisição?']
                  return random.choice(responses).replace('nome',self.contextos['Nome'])
              else:
                intencao = [intent for intent in self.dados['intents'] if intent['tag']== 'encerramento']
                responses = self.decide_respostas_tempo(intencao[0])
                return random.choice(responses).replace('nome',self.contextos['Nome'])
            else:
              if tag == 'encerramento':
                self.muda_status()
                intencao = [intent for intent in self.dados['intents'] if intent['tag'] == 'encerramento']
                responses = self.decide_respostas_tempo(intencao[0])
                return random.choice(responses).replace('nome',self.contextos['Nome'])
              return "Por favor, me ajude na conversa, ainda estou aprendendo"
          
          # incidente  
          elif self.estados['incidente']:
            results_inc = self.model_inc.predict([self.bag_of_words(unidecode(mensagem), self.words_inc)])
            idx_inc = np.argmax(results_inc)
            tag_inc = self.labels_inc[idx_inc]
            if results_inc[0][idx_inc] > 0.7:
              for tg in self.dados_inc["intents"]:
                if tg['Tag'] == tag_inc:
                  if tg['TextoAutomacao']: #tem acao
                    #resposta_acao = unlock_user(996801134)
                    responses = tg['TextoAutomacao'] #resposta_acao if resposta_acao else tg['TextoAutomacao']                                                
                  else:
                    responses = tg['RespostaSelfService']
            else:
              responses=['Procurei no FAQ e não encontrei. Poderia reescrever?']
            return random.choice(responses)
          
          # requisicao 
          elif self.estados['requisicao']:
            results_req = self.model_req.predict([self.bag_of_words(unidecode(mensagem), self.words_req)])
            idx_req = np.argmax(results_req)
            tag_req = self.labels_req[idx_req]
            if results_req[0][idx_req] > 0.7:
              for tg in self.dados_req["intents"]:
                if tg['Tag'] == tag_req:
                  if tg['TextoAutomacao']: #tem acao
                    #resposta_acao = change_password(996801134)
                    responses = tg['TextoAutomacao']  #resposta_acao if resposta_acao else tg['TextoAutomacao']                                                
                  else:
                    responses = tg['RespostaSelfService']
            else:
              responses=['Procurei no FAQ e não encontrei. Poderia reescrever?']
            return random.choice(responses)
          else:
            return "Desculpa não entendi, poderia digitar de outra forma?"

    def responder(self, resposta, chat_id):
        
        '''
        if resposta == '1':
          reply_markup = ReplyKeyboardMarkup(KeyboardButton('Compartilhar telefone?', request_contact=True))        
          self.bot.sendMessage(chat_id, resposta, reply_markup=reply_markup)
        else:
          self.bot.sendMessage(chat_id, resposta)
        '''
        link_requisicao = f'{self.url_base}sendMessage?chat_id={chat_id}&text={resposta}'
        requests.get(link_requisicao)
    
    
    def grava(self, obj, filename):  
        with open(f'{filename}.pickle', 'wb') as pkl_file:
          pickle.dump(obj,pkl_file)
    
    def abre(self, filename):
        with open(f'{filename}.pickle', 'rb') as pkl_file:
          return pickle.load(pkl_file)
    
    def estrutura_rede(self, train, output, neuronios):
        ops.reset_default_graph()

        net = tflearn.input_data(shape=[None, len(train[0])])
        net = tflearn.fully_connected(net, neuronios)
        net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
        net = tflearn.regression(net)

        return tflearn.DNN(net)

    def carregar_dados(self, nome_model, neuronios=4):
      nome = nome_model+".pickle"
      with open(path_dev+nome, "rb") as f:
          words, labels, training, output = pickle.load(f)
      model_name =  "./"+nome_model+".tflearn"
      if nome_model == 'Requisicao':
        model = self.estrutura_rede(training, output, 8)
      else:
        model = self.estrutura_rede(training, output, neuronios) 
      model.load(path_dev+model_name)
      return [model, words, labels]

    def bag_of_words(self, s, words):
      bag = [0 for _ in range(len(words))]
      s_words = nltk.word_tokenize(s)
      s_words = [stemmer.stem(word.lower()) for word in s_words]

      for se in s_words:
        for i, w in enumerate(words):
          if w == se:
            bag[i] = 1
      return np.array(bag)
    
    @staticmethod
    def limpa_input(input):
      '''
      remove pontuação e acentos
      '''
      new_input = unidecode.unidecode(input.lower())
      new_input = ''.join(re.findall('\d+\.\d+|\d+\,\d+|\w|\s', new_input))
      
      return new_input

    def run_once(f):
      def wrapper(*args, **kwargs):
          if not wrapper.has_run:
              wrapper.has_run = True
              return f(*args, **kwargs)
      wrapper.has_run = False
      return wrapper
  
    @run_once
    def obtem_tempo(self):
      hora = datetime.datetime.now().hour
      if hora < 12:
        tempo = 'manha'
      elif hora > 12 and hora < 18:
        tempo = 'tarde'
      else:
        tempo = 'noite'

      return tempo

In [ ]:
'''
                elif (tg['tag'] == tag and tag == 'identificacao') or status == 'identificacao':
                  responses = tg['responses']
                  self.estados['identificacao'] = False
                  for ent in nlp(mensagem).ents:
                    if ent.label == 'PER':
                      self.contextos['Nome'] = ent.text              
                    return random.choice(responses).replace('nome',self.contextos['Nome'])
'''

### mlp

In [ ]:
class ModeloChatBot():

    def __init__(self, dados):
      self.words = []
      self.labels = []
      self.docs_x = []
      self.docs_y = []
      self.training=[]
      self.output=[]
      self.data = dados
    
    @staticmethod
    def grava(obj, filename):  
      with open(f'{filename}.pkl', 'wb') as pkl_file:
        pickle.dump(obj,pkl_file)
    
    @staticmethod
    def abre(filename):
      with open(f'{filename}.pkl', 'rb') as pkl_file:
        return pickle.load(pkl_file)

    def processa_intents(self):
      '''
      '''
      for intent in self.data['intents']:
        for pattern in intent['patterns']:
          wrds = nltk.word_tokenize(pattern)
          self.words.extend(wrds)
          self.docs_x.append(wrds)
          self.docs_y.append(intent['tag'])

        if intent['tag'] not in self.labels:
          self.labels.append(intent['tag'])
    
    def estrutura_rede(self, train, out, neuronios=4):
        ops.reset_default_graph()

        net = tflearn.input_data(shape=[None, len(train)])
        net = tflearn.fully_connected(net, neuronios)
        net = tflearn.fully_connected(net, len(out), activation="softmax")
        net = tflearn.regression(net)

        return tflearn.DNN(net)

    def preprocessamento(self):
      '''
      '''
      self.words = [stemmer.stem(unidecode(w.lower())) for w in self.words if w != "?"]
      self.words = sorted(list(set(self.words)))
      self.labels = sorted(self.labels)
      
      out_empty = [0 for _ in range(len(self.labels))]

      for x, doc in enumerate(self.docs_x):
        bag = []
        wrds = [stemmer.stem(unidecode(w.lower())) for w in doc]
        for w in self.words:
          if w in wrds:
            bag.append(1)
          else:
            bag.append(0)
        output_row = out_empty[:]
        output_row[self.labels.index(self.docs_y[x])] = 1
        
        self.training.append(bag)
        self.output.append(output_row)

      self.training = np.array(self.training)
      self.output = np.array(self.output)
      
      with open(path_dev+'model.pickle', "wb") as f:
        pickle.dump((self.words, self.labels, self.training, self.output), f)
    
    def treina_modelo(self, epocas):
      self.processa_intents()
      self.preprocessamento()
      
      model = self.estrutura_rede(self.training[0], self.output[0])
      model.fit(self.training, self.output, n_epoch=epocas, batch_size=8, show_metric=True)
      model.save(path_dev+"model.tflearn")


In [ ]:
model = ModeloChatBot(dados).treina_modelo(500)

Training Step: 3499  | total loss: 0.46236 | time: 0.029s
| Adam | epoch: 500 | loss: 0.46236 - acc: 0.9344 -- iter: 48/49
Training Step: 3500  | total loss: 0.41841 | time: 0.033s
| Adam | epoch: 500 | loss: 0.41841 - acc: 0.9410 -- iter: 49/49
--
INFO:tensorflow:/content/drive/My Drive/Residencia/bot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### run

In [ ]:
bot = TelegramBot(dados, dados_req, dados_inc)
bot.Iniciar()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/bot/./Requisicao.tflearn
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/bot/./Incidentes.tflearn
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/bot/./model.tflearn
saudacao
incidente
incidente
incidente
encerramento
saudacao
saudacao
saudacao
saudacao
saudacao
saudacao
incidente
incidente
incidente
requisicao
requisicao
saudacao
incidente
incidente
incidente
requisicao
saudacao
saudacao
requisicao
requisicao
requisicao
requisicao
requisicao
requisicao
requisicao
saudacao
requisicao
requisicao
saudacao
incidente
incidente
incidente
requisicao
saudacao
saudacao
incidente
incidente
incidente
requisicao
saudacao
incidente
incidente
incidente
requisicao
saudacao
saudacao
incidente
incidente
requisicao
saudacao
incidente
incidente
incidente
incidente
requisicao


KeyboardInterrupt: ignored